In [1]:
import pandas as pd
from web_scrapping import web_scrape

In [9]:
drug_data = pd.read_csv("/Users/shobanasiranjeevilu/Downloads/Healthcare_LLM/drug_data.csv")

In [10]:
diseases = drug_data.Prescribed_for.unique()

In [11]:
# considering subset of data
subset_of_diseases = list(diseases[236:472])

In [12]:
len(subset_of_diseases)

236

In [21]:
# clenaing the unnecesary disease type is yet to be processed

disease_list = []
for disease in subset_of_diseases:
  if "</span>" not in disease:
    disease_list.append(disease)


In [16]:
#  only considering drugs which has medium to high rating
filtered_df = drug_data[(drug_data['Prescribed_for'].isin(disease_list)) & (drug_data['User_Rating']>=5)]

In [17]:
aggregated_df = filtered_df.groupby('Prescribed_for').agg(DrugName_list=('drugName', 'unique')).reset_index(drop=False)


In [18]:
aggregated_df

,Prescribed_for,DrugName_list
0,AIDS Related Wasting,[Serostim]
1,Abortion,[Misoprostol]
2,Acute Coronary Syndrome,"[Brilinta, Ticagrelor, Prasugrel]"
3,Acute Lymphoblastic Leukemia,[Sprycel]
4,Agitated State,"[Aripiprazole, Abilify]"
...,...,...
213,min),"[Metformin, Glyburide / metformin, Dapaglifloz..."
214,moterol / mometasone),[Formoterol / mometasone]
215,moterol),"[Budesonide / formoterol, Formoterol, Arformot..."
216,von Willebrand's Disease,[Desmopressin]


In [22]:
diseases_final = list(aggregated_df['Prescribed_for'])

### Web scraping to retrive symptoms for subset of diseases

In [23]:
disease_with_symptoms = web_scrape(diseases_final)

In [30]:
prescribed_for, symptoms = [], []
for key, value in disease_with_symptoms.items():
    prescribed_for.append(key)
    symptoms.append(value)


In [33]:
symptoms_df = pd.DataFrame({'Prescribed_for': prescribed_for, 'Symptoms': symptoms}, index=range(1, len(disease_with_symptoms) + 1))

In [39]:
subset_df = aggregated_df.merge(symptoms_df,on='Prescribed_for', how='left')